In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
import numpy as np

In [ ]:

file_path = 'E:\MS_AI\BKW_hack\BKW-Design-Agent\ML_model\data\extracted_historical_data_04.csv'

try:
    data = pd.read_csv(file_path)
    print("CSV file loaded successfully!")
    display(data.head()) # Display the first few rows to confirm
except FileNotFoundError:
    print(f"Error: The file was not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
data.shape

In [ ]:
import joblib
df = pd.DataFrame(data)

# --- 2. Feature Selection and Preprocessing ---
features = ['L (mm)', 'Material', 'h (mm)', 'w (mm)', 'F (N)']
target = 'Deflection (mm)'
X = df[features]
y = df[target]

label_encoder = LabelEncoder()
# FIX #1: Use .loc to avoid the SettingWithCopyWarning
X.loc[:, 'Material'] = label_encoder.fit_transform(X['Material'])

# --- 3. Splitting and Scaling ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the scaler ONLY on the training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
# Transform the test data using the same scaler
X_test_scaled = scaler.transform(X_test)

# --- 4. Training the Model ---
# The model is trained on the scaled data
model = RandomForestRegressor(n_estimators=400, random_state=42)

# model = RandomForestRegressor(
#     n_estimators=400,          # 400 is a good number of trees, we can keep it.
#     max_depth=15,              # *** MOST IMPORTANT ***: Prevents trees from growing too deep.
#     min_samples_leaf=5,        # *** IMPORTANT ***: Ensures each final prediction is based on at least 5 samples.
#     max_features='sqrt',       # *** IMPORTANT ***: Reduces correlation between trees by using a subset of features.
#     random_state=42,
#     n_jobs=-1                  # Use all available CPU cores to speed up training.
# )
model.fit(X_train_scaled, y_train)

# --- 5. Thorough Performance Evaluation ---
print("--- Model Performance Evaluation ---")
y_pred_test = model.predict(X_test_scaled)
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
print(f"Test Set Mean Absolute Error (MAE): {mae_test:.2f} mm")
print(f"Test Set Root Mean Squared Error (RMSE): {rmse_test:.2f} mm")
print(f"Test Set Mean Squared Error (MSE): {mse_test:.2f}")

y_pred_train = model.predict(X_train_scaled)
mse_train = mean_squared_error(y_train, y_pred_train)
print(f"Train Set Mean Squared Error on Training Data: {mse_train:.2f}")

# --- 6. Saving the Trained Model and Preprocessors ---
print("\n--- Saving Model and Preprocessors ---")
joblib.dump(model, 'random_forest_deflection_model.joblib')
joblib.dump(scaler, 'scaler.joblib')
joblib.dump(label_encoder, 'label_encoder.joblib')
print("Model, scaler, and label encoder saved successfully.")


In [ ]:
# --- 7. Loading the Saved Model and Preprocessors ---
print("\n--- Loading Model and Preprocessors for Inference ---")
loaded_model = joblib.load('random_forest_deflection_model.joblib')
loaded_scaler = joblib.load('scaler.joblib')
loaded_label_encoder = joblib.load('label_encoder.joblib')
print("Assets loaded successfully.")

# --- 8. Prediction Function using LOADED Assets ---
def predict_with_loaded_model(length, material, height, width, force):
    """
    Predicts deflection using the loaded model and preprocessors.
    """
    new_data = pd.DataFrame({
        'L (mm)': [length], 'Material': [material], 'h (mm)': [height], 'w (mm)': [width], 'F (N)': [force]
    })
    # Use the loaded encoder and scaler
    new_data['Material'] = loaded_label_encoder.transform(new_data['Material'])
    new_data_scaled = loaded_scaler.transform(new_data)
    
    # Predict using the loaded model
    predicted_deflection = loaded_model.predict(new_data_scaled)[0]
    
    allowable_deflection = length / 240
    status = "PASS" if predicted_deflection < allowable_deflection else "FAIL"

    print(f"\n--- Prediction using LOADED Model ---")
    print(f"Input Length: {length} mm")
    print(f"Predicted Deflection: {predicted_deflection:.2f} mm")
    print(f"Allowable Deflection: {allowable_deflection:.2f} mm")
    print(f"Status: {status}")

predict_with_loaded_model(length=2500, material='Steel', height=80, width=46, force=10000)
predict_with_loaded_model(length=1200, material='Steel', height=80, width=46, force=10000)

In [ ]:
r2 = r2_score(y_test, y_pred_test)
print(f"R-squared (R²) Score: {r2:.4f}")

mae_test = mean_absolute_error(y_test, y_pred_test)
print(f"Mean Absolute Error (MAE): {mae_test:.2f} mm")
print("\n--- 'Proxy' Pass/Fail Accuracy ---")

# To calculate accuracy, we need the original unscaled Lengths from the test set
X_test_unscaled = scaler.inverse_transform(X_test_scaled)
X_test_unscaled_df = pd.DataFrame(X_test_unscaled, columns=X_train.columns)

# Determine "Actual Status" and "Predicted Status"
allowable_deflection = X_test_unscaled_df['L (mm)'] / 240

# FIX: Use .values to compare the underlying numpy arrays, ignoring the pandas index
actual_status = np.where(y_test.values < allowable_deflection.values, 'PASS', 'FAIL')
# y_pred_test is already a numpy array, so we only need .values on allowable_deflection
predicted_status = np.where(y_pred_test < allowable_deflection.values, 'PASS', 'FAIL')

# Calculate the accuracy of the final Pass/Fail decision
pass_fail_accuracy = accuracy_score(actual_status, predicted_status)
print(f"Pass/Fail Decision Accuracy: {pass_fail_accuracy:.2%}")

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score

def evaluate_model(test_data):
    """
    Loads a pre-trained model and its preprocessors to evaluate performance
    on a new, unseen dataset.

    Args:
        data_filepath (str): The path to the new CSV data file.
    """
    # --- 1. Load Pre-trained Assets ---
    print("--- Loading Pre-trained Model and Preprocessors ---")
    try:
        model = joblib.load('random_forest_deflection_model.joblib')
        scaler = joblib.load('scaler.joblib')
        label_encoder = joblib.load('label_encoder.joblib')
        print("Assets loaded successfully.")
    except FileNotFoundError as e:
        print(f"Error: Could not find a required file. Make sure model, scaler, and encoder files are in the same directory.")
        print(f"Details: {e}")
        return # Exit the function if files are missing

    new_df = test_data
    
    

    # Define the expected feature and target columns
    features = ['L (mm)', 'Material', 'h (mm)', 'w (mm)', 'F (N)']
    target = 'Deflection (mm)'

    # Ensure all required columns are present in the new data
    required_columns = features + [target]
    if not all(col in new_df.columns for col in required_columns):
        print(f"Error: The new data file is missing one or more required columns. Required: {required_columns}")
        return

    X_new = new_df[features].copy()
    y_new = new_df[target]

    # --- 3. Preprocess New Data using LOADED Assets ---
    # IMPORTANT: Use the loaded preprocessors to transform the data. DO NOT refit them.
    
    # a. Encode the 'Material' column
    try:
        X_new['Material'] = label_encoder.transform(X_new['Material'])
    except ValueError as e:
        print(f"Error: Could not encode the 'Material' column. It may contain a material not seen during training.")
        print(f"Details: {e}")
        return

    # b. Scale the features
    X_new_scaled = scaler.transform(X_new)

    # --- 4. Make Predictions ---
    print("\n--- Making Predictions on New Data ---")
    y_pred_new = model.predict(X_new_scaled)
    print("Predictions generated successfully.")

    # --- 5. Evaluate and Report Performance ---
    print("\n--- Performance Report on New Data ---")

    # a. Regression Metrics
    r2 = r2_score(y_new, y_pred_new)
    mae = mean_absolute_error(y_new, y_pred_new)
    rmse = np.sqrt(mean_squared_error(y_new, y_pred_new))

    print(f"R-squared (R²) Score: {r2:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.2f} mm")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f} mm")

    # b. 'Proxy' Pass/Fail Accuracy
    allowable_deflection = X_new['L (mm)'] / 240
    actual_status = np.where(y_new.values < allowable_deflection.values, 'PASS', 'FAIL')
    predicted_status = np.where(y_pred_new < allowable_deflection.values, 'PASS', 'FAIL')
    
    pass_fail_accuracy = accuracy_score(actual_status, predicted_status)
    print(f"\n'Proxy' Pass/Fail Decision Accuracy: {pass_fail_accuracy:.2%}")


# --- Main execution block ---
if __name__ == "__main__":
    file_path = r"E:\MS_AI\BKW_hack\BKW-Design-Agent\ML_model\data\extracted_historical_data_00.csv"
    try:
        test_data = pd.read_csv(file_path)
        print("CSV file loaded successfully!")
    except FileNotFoundError:
        print(f"Error: The file was not found at {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")
    
    # Run the evaluation
    evaluate_model(test_data)

to return filtered excel based on length, material type, status

In [ ]:
import pandas as pd

def filter_min_volume_pass(output_excel):
    """
    Reads beam data, filters only rows with PASS status, and for each (L, Material)
    keeps the row with the minimum Volume (V).
    Saves the filtered result to a new CSV file.
    """
    file_path = '/content/drive/MyDrive/extracted_historical_data_00.csv'

    try:
        df = pd.read_csv(file_path)
        print(df.shape)
    except FileNotFoundError:
        print(f"Error: The file was not found at {file_path}")

    # Normalize column names
    df.columns = df.columns.str.strip()

    # Filter only PASS rows
    df_pass = df[df['Status'].str.upper() == 'PASS']

    # Select minimum volume per (L (mm), Material)
    df_min_vol = (
        df_pass.loc[
            df_pass.groupby(['L (mm)', 'Material'])['V (mm^3)'].idxmin()
        ]
        .sort_values(['L (mm)', 'Material'])
    )

    # Save to new CSV
    # df_min_vol.to_excel(output_excel, index=False)
    print(f"Filtered Excel file saved to: {output_excel}")
    


returns Json and text files, for material and length input

In [ ]:
import pandas as pd
import json

def get_beam_json_and_row(excel_path, material, length, json_out, text_out):
    """
    Reads the filtered Excel file and returns:
      1. JSON file containing key parameters (Material, L, h, w, F)
      2. Text file containing the entire row data

    Args:
        excel_path (str): Path to Excel file
        material (str): Material type to search
        length (int or float): Beam length
        json_out (str): Path to save JSON output
        text_out (str): Path to save text file output
    """
    df = pd.read_excel(excel_path)
    df.columns = df.columns.str.strip()

    row = df[(df['Material'] == material) & (df['L (mm)'] == length)]

    if row.empty:
        msg = f"No matching entry found for Material={material}, L={length}"
        with open(text_out, "w") as f:
            f.write(msg)
        return json.dumps({"error": msg}, indent=4)

    # Extract required fields for JSON
    data_json = row.iloc[0][['Material', 'L (mm)', 'h (mm)', 'w (mm)', 'F (N)']].to_dict()

    # Save JSON file
    with open(json_out, "w") as f:
        json.dump(data_json, f, indent=4)

    # Save entire row to text file
    with open(text_out, "w") as f:
        f.write(row.to_string(index=False))

    print(f"JSON saved to: {json_out}")
    print(f"Full row saved to: {text_out}")

    return data_json

get_beam_json_and_row(
    "beam_data_filtered.xlsx",
    "Steel",
    1500,
    "beam_steel_1500.json",
    "beam_steel_1500.txt"
)
